In [1]:
#https://github.com/keras-team/keras/releases
!pip install q keras==2.11.0
!pip install tensorflow==2.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.11.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import keras
from keras import layers 
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam, SGD

In [3]:
print("Tensorflow version " + tf.__version__)
np.random.seed(123)
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.9.2
Running on TPU  ['10.5.187.146:8470']


## Loading the dataset.

In [31]:
c = pd.read_csv('/content/final.csv', header=None)

In [9]:
data

,0,1,2,3,4,5,6
0,76561197970982479,1250,1,76561197970982479,10006,0,0
1,76561197970982479,22200,1,76561197970982479,271,0,0
2,76561197970982479,43110,1,76561197970982479,834,0,0
3,js41637,227300,1,76561198035864385,551,0,1
4,js41637,239030,1,76561198035864385,349,0,1
...,...,...,...,...,...,...,...
40310,Ghoustik,730,1,76561198308105615,3969,0,18739
40311,76561198312638244,233270,1,76561198312638244,533,0,18740
40312,76561198312638244,130,1,76561198312638244,221,0,18740
40313,76561198312638244,70,1,76561198312638244,1010,0,18740


## Normalizing the data.

In [11]:
# example of a standardization
from numpy import asarray
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [12]:
data[[4, 5]] = scaler.fit_transform(data[[4, 5]])
data

,0,1,2,3,4,5,6
0,76561197970982479,1250,1,76561197970982479,-0.003969,-0.194517,0
1,76561197970982479,22200,1,76561197970982479,-0.430972,-0.194517,0
2,76561197970982479,43110,1,76561197970982479,-0.406277,-0.194517,0
3,js41637,227300,1,76561198035864385,-0.418691,-0.194517,1
4,js41637,239030,1,76561198035864385,-0.427551,-0.194517,1
...,...,...,...,...,...,...,...
40310,Ghoustik,730,1,76561198308105615,-0.268768,-0.194517,18739
40311,76561198312638244,233270,1,76561198312638244,-0.419480,-0.194517,18740
40312,76561198312638244,130,1,76561198312638244,-0.433165,-0.194517,18740
40313,76561198312638244,70,1,76561198312638244,-0.398558,-0.194517,18740


## Constructing and splitting the dataset.

In [13]:
X = data[[6, 1, 4, 5]].values #,
y = data[2]

In [28]:
type(X)

numpy.ndarray

In [14]:
X

array([[ 0.00000000e+00,  1.25000000e+03, -3.96915275e-03,
        -1.94516796e-01],
       [ 0.00000000e+00,  2.22000000e+04, -4.30972069e-01,
        -1.94516796e-01],
       [ 0.00000000e+00,  4.31100000e+04, -4.06277396e-01,
        -1.94516796e-01],
       ...,
       [ 1.87400000e+04,  1.30000000e+02, -4.33165201e-01,
        -1.94516796e-01],
       [ 1.87400000e+04,  7.00000000e+01, -3.98557569e-01,
        -1.94516796e-01],
       [ 1.87400000e+04,  3.62890000e+05, -3.47062816e-01,
        -1.94516796e-01]])

In [15]:
y

0        1
1        1
2        1
3        1
4        1
        ..
40310    1
40311    1
40312    1
40313    1
40314    1
Name: 2, Length: 40315, dtype: int64

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y,stratify=y, test_size=0.25)

In [17]:
X_train.shape

(30236, 4)

In [18]:
y_train

35745    1
19918    1
11033    1
32941    1
12755    1
        ..
9212     1
30402    1
1046     1
26077    1
29262    1
Name: 2, Length: 30236, dtype: int64

In [19]:
X_train_array = [X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3]]
X_val_array = [X_val[:, 0], X_val[:, 1], X_val[:, 2], X_val[:, 3]]

## Model building, with 2 weeks play hours and total play houurs as meta data.

In [24]:
in_u_dim = len(data[6].unique())
in_g_dim = len(data[1].unique())
 
latent_out_dim = 8 #embedding size

In [25]:
# Embedding layers
from keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten
from keras import regularizers

def ncf(): 
    
    user_gmf = Input(name = 'u_in_gmf', shape = [1])
    game_gmf = Input(name = 'b_in_gmf', shape = [1])

    tp_gmf = Input(name = 'tp_in_gmf', shape = [1])
    bwp_gmf = Input(name = 'bwp_in_gmf', shape = [1])

    user_embedding_gmf = Embedding(name = 'u_emb_gmf', embeddings_regularizer=regularizers.L2(0), input_dim = in_u_dim, output_dim = latent_out_dim)(user_gmf)
    game_embedding_gmf = Embedding(name = 'g_emb_gmf', embeddings_regularizer=regularizers.L2(0), input_dim = in_g_dim, output_dim = latent_out_dim)(game_gmf)
    #meta-----------------------------------------------------------------------------------------------------------------
    tp_embedding_gmf = Embedding(name = 'tp_emb_gmf', embeddings_regularizer=regularizers.L2(0), input_dim = in_u_dim, output_dim = latent_out_dim)(tp_gmf)
    bwp_embedding_gmf = Embedding(name = 'bwp_emb_gmf', embeddings_regularizer=regularizers.L2(0), input_dim = in_u_dim, output_dim = latent_out_dim)(bwp_gmf)
    #----------------------------------------------------------------------------------------------------------------------

   # x = tf.keras.layers.Multiply()([user_embedding_gmf, game_embedding_gmf,age_embedding_gmf])
   # x = Model(inputs=[user_gmf,game_gmf,age_gmf], outputs=x)
    x = tf.keras.layers.Multiply()([user_embedding_gmf, game_embedding_gmf, tp_embedding_gmf, bwp_embedding_gmf])
    x = Model(inputs=[user_gmf,game_gmf,tp_gmf, bwp_gmf], outputs=x)


    user_mlp = Input(name = 'u_in_mlp', shape = [1])
    game_mlp = Input(name = 'g_in_mlp', shape = [1])
    tp_mlp = Input(name = 'tp_in_mlp', shape = [1])
    bwp_mlp = Input(name = 'bwp_in_mlp', shape = [1])
    # age_mlp = Input(name = 'a_in_mlp', shape = [1])


    user_embedding_mlp = Embedding(name = 'u_emb_mlp', embeddings_regularizer=regularizers.L2(0), input_dim = in_u_dim,output_dim = latent_out_dim)(user_mlp)
    game_embedding_mlp = Embedding(name = 'g_emb_mlp', embeddings_regularizer=regularizers.L2(0), input_dim = in_g_dim, output_dim = latent_out_dim)(game_mlp)
 
    tp_embedding_mlp = Embedding(name = 'tp_emb_mlp', embeddings_regularizer=regularizers.L2(0), input_dim = in_g_dim, output_dim = latent_out_dim)(tp_mlp)
    bwp_embedding_mlp = Embedding(name = 'bwp_emb_mlp', embeddings_regularizer=regularizers.L2(0), input_dim = in_g_dim, output_dim = latent_out_dim)(bwp_mlp)

    #meta-----------------------------------------------------------------------------------------------------------------
    # age_embedding_mlp = Embedding(name = 'a_emb_mlp', embeddings_regularizer=regularizers.L2(0), input_dim = in_u_dim, output_dim = latent_out_dim)(age_mlp)
    #----------------------------------------------------------------------------------------------------------------------


    y = tf.keras.layers.Concatenate()([user_embedding_mlp, game_embedding_mlp, tp_embedding_mlp, bwp_embedding_mlp])



   # y = tf.keras.layers.Concatenate()([user_embedding_mlp, game_embedding_mlp,age_embedding_mlp])
    y = Flatten()(y)

    y = keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L2(0))(y) 
    y = keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.L2(0))(y) 
    y = keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizers.L2(0))(y) 
    y = keras.layers.Dense(8, activation='relu', kernel_regularizer=regularizers.L2(0))(y) 
    y = keras.layers.Reshape([1, 8])(y)
    #y = Model(inputs=[user_mlp,game_mlp,age_mlp], outputs=y)


    y = Model(inputs=[user_mlp,game_mlp, tp_mlp, bwp_mlp], outputs=y)



    combined = tf.keras.layers.Concatenate()([x.output, y.output])
    z = Dense(1, kernel_initializer='lecun_uniform')(combined)
    z = Activation("sigmoid")(z)
  
    model = Model(inputs=[[user_gmf,game_gmf, tp_gmf, bwp_gmf], [user_mlp, game_mlp, tp_mlp, bwp_mlp]], outputs=z)
#    model = Model(inputs=[[user_gmf,game_gmf,age_gmf], [user_mlp,game_mlp,age_mlp]], outputs=z)
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    return model

with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model = ncf()
model.summary()
#https://colab.research.google.com/notebooks/tpu.ipynb#scrollTo=hJl3vNtJOB-x

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 u_in_mlp (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 g_in_mlp (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 tp_in_mlp (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 bwp_in_mlp (InputLayer)        [(None, 1)]          0           []                               
                                                                                            

## Model training

In [26]:
from tensorflow.core.protobuf.tpu.optimization_parameters_pb2 import LearningRate
# Model training
batch_size = 2048

history = model.fit(
    x=[X_train_array,X_train_array],
    y=[y_train,y_train],
    batch_size=batch_size,
    epochs=7,
    verbose=1,
    validation_data=([X_val_array, X_val_array], [y_val, y_val])
)



Epoch 1/7
15/15 [==============================] - 23s 895ms/step - loss: 0.4538 - root_mean_squared_error: 0.6737 - val_loss: 0.4519 - val_root_mean_squared_error: 0.6722
Epoch 2/7
15/15 [==============================] - 2s 168ms/step - loss: 0.4501 - root_mean_squared_error: 0.6709 - val_loss: 0.4475 - val_root_mean_squared_error: 0.6689
Epoch 3/7
15/15 [==============================] - 1s 83ms/step - loss: 0.4436 - root_mean_squared_error: 0.6660 - val_loss: 0.4372 - val_root_mean_squared_error: 0.6612
Epoch 4/7
15/15 [==============================] - 1s 89ms/step - loss: 0.4265 - root_mean_squared_error: 0.6531 - val_loss: 0.4097 - val_root_mean_squared_error: 0.6401
Epoch 5/7
15/15 [==============================] - 1s 76ms/step - loss: 0.3871 - root_mean_squared_error: 0.6222 - val_loss: 0.3685 - val_root_mean_squared_error: 0.6071
Epoch 6/7
15/15 [==============================] - 1s 101ms/step - loss: 0.3611 - root_mean_squared_error: 0.6009 - val_loss: 0.3657 - val_root_mea

In [27]:
model.save('NCF_with_meta.h5')

## Predicting games for user

In [54]:
import csv
reader = csv.reader(open('/content/user_id_map.csv', 'r'))
user_map = {}
for row in reader:
   k, v = row
   user_map[k] = v

In [61]:
reader = csv.reader(open('/content/item_id_game_name_map.csv', 'r'))
game_map = {}
for row in reader:
   k, v = row
   game_map[k] = v

In [33]:
from scipy.sparse import data
e = 0
final_game_set = set()

for index, row in c.iterrows():
    try:
        # print(str(row[1]))
        final_game_set.add(row[1])
    except:
        e = e + 1
        pass

In [66]:
def get_top_n(n, username, model):
  input_X = pd.DataFrame(final_game_set)
  input_X[1] = int (user_map[username])
  input_X[2] = 0 
  input_X[3] = 0
  
  X_input = input_X[[1, 0, 2, 3]].values
  X_input = [X_input[:, 0], X_input[:, 1], X_input[:, 2], X_input[:, 3]]
  test_pred = model.predict([X_input, X_input])

  l = test_pred.tolist()
  tmp = []

  for l1 in l:
    for l2 in l1:
      for l3 in l2:
        tmp.append(l3)

  input_X['score'] = tmp
  game_ids = input_X.sort_values('score', ascending = False)[0: n][0].tolist()
  for game in game_ids:
    print(game_map[str(game)])



## Testing

In [67]:
get_top_n(5, 'thefreak9', model)


29/29 [==============================] - 1s 15ms/step
Counter-Strike: Source
Portal
Left 4 Dead 2
Half-Life
Portal 2
